# Final Assignment - part 3

## First stage - load the data 

In [1]:
import pandas as pd
import pickle
import unicodedata
df = pd.read_csv('data_final.csv',header=0, 
                    delimiter=",")
#df = df.ix[1:]
#df = pd.DataFrame(data=df, columns=['text','type'])





## Cleaning the data

In [2]:
vocabulary_size = 300
unknown_token = "UNKNOWNTOKEN"
sentence_start_token = "SENTENCESTART"
sentence_end_token = "SENTENCEEND"
line_break= "NEWLINE"
separator= "SEPARATOR"

In [3]:
for d in df["type"]:
    text1 = d.replace('\n',' '+ line_break + ' ')
    text1 = text1.replace('--',' '+ separator + ' ')
    text1 = text1.replace('.',' '+sentence_end_token +' '+ sentence_start_token+' ' )
df["type"][2] 



'afromarengo plana jumping spider species genus afromarengo lives south africa'

# Data Preparing

## Split the data into 3 different structures to build the 3 models

In [4]:
wiki_data   = df[df["text"] == "wiki"]
cnn_data    = df[df["text"] == "cnn"]
recipe_data = df[df["text"] == "recipe"]

In [5]:
import numpy as np
import theano
import keras 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import text_to_word_sequence

wiki_string = ""
for wk in wiki_data["type"]:
    wiki_string+= " " + wk
wiki_words = text_to_word_sequence(wiki_string, lower=False, split=" ")

cnn_string = ""
for cn in cnn_data["type"]:
    cnn_string+= " " + cn
cnn_words = text_to_word_sequence(cnn_string, lower=False, split=" ")


recipe_string = ""
for rc in recipe_data["type"]:
    recipe_string+= " " + rc
recipe_words = text_to_word_sequence(recipe_string, lower=False, split=" ")


Using Theano backend.


## Using sklearn vectorizer to create productive vocablury for 3 different list of words(cnn,wiki,recipe)
## Make list of lists that for every 5 words knows the 6 one
## Using keras library to create sequence of word from text

In [6]:
from keras.preprocessing.text import text_to_word_sequence
from sklearn.feature_extraction.text import CountVectorizer
xData = []
yData = []
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features=600
                              ) 
vectorizer.fit_transform(wiki_words)
vocab = vectorizer.get_feature_names()
vocab.append("unknown")
vocab_words = []
for w in wiki_words:
    if w in vocab:
        vocab_words.append(w)
    else:
        vocab_words.append("unknown")
convert_words_to_int = dict()
convert_int_to_words = dict()
for i, c in enumerate(vocab):
    convert_words_to_int[c] = i
for i, c in enumerate(vocab):
    convert_int_to_words[i] = c
num_of_words = len(vocab_words)
num_of_vocav = len(vocab)

for i in range(0, num_of_words - 5, 1):
        list_in = vocab_words[i:i + 5]
        list_out = vocab_words[i + 5]
        xData.append([convert_words_to_int[word] for word in list_in])
        yData.append(convert_words_to_int[list_out])


In [7]:
xData2 = []
yData2 = []
vectorizer2 = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features=600
                              ) 
vectorizer2.fit_transform(cnn_words)
vocab2 = vectorizer2.get_feature_names()
vocab2.append("unknown")
vocab_words2 = []
for w in cnn_words:
    if w in vocab2:
        vocab_words2.append(w)
    else:
        vocab_words2.append("unknown")
convert_words_to_int2 = dict()
convert_int_to_words2 = dict()
for i, c in enumerate(vocab2):
    convert_words_to_int2[c] = i
for i, c in enumerate(vocab2):
    convert_int_to_words2[i] = c
num_of_words2 = len(vocab_words2)
num_of_vocav2 = len(vocab2)

for i in range(0, num_of_words2 - 5, 1):
        list_in2 = vocab_words2[i:i + 5]
        list_out2 = vocab_words2[i + 5]
        xData2.append([convert_words_to_int2[word] for word in list_in2])
        yData2.append(convert_words_to_int2[list_out2])

In [8]:
xData3 = []
yData3 = []
vectorizer3 = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features=600
                              ) 
vectorizer3.fit_transform(recipe_words)
vocab3 = vectorizer3.get_feature_names()
vocab3.append("unknown")
vocab_words3 = []
for w in recipe_words:
    if w in vocab3:
        vocab_words3.append(w)
    else:
        vocab_words3.append("unknown")
convert_words_to_int3 = dict()
convert_int_to_words3 = dict()
for i, c in enumerate(vocab3):
    convert_words_to_int3[c] = i
for i, c in enumerate(vocab3):
    convert_int_to_words3[i] = c
num_of_words3 = len(vocab_words3)
num_of_vocav3 = len(vocab3)

for i in range(0, num_of_words3 - 5, 1):
        list_in3 = vocab_words3[i:i + 5]
        list_out3 = vocab_words3[i + 5]
        xData3.append([convert_words_to_int3[word] for word in list_in3])
        yData3.append(convert_words_to_int3[list_out3])

## Create 3 models for each kind of data
## The model we chose is the simple RNN model
### 5.Create a model with:
### an Embedding input layer with 42 entries
### a SimpleRNN layer with 186 hidde nodes and a relu activation function and identity initialization
### a Dense output layer with a softmax activation function

In [9]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.recurrent import SimpleRNN
from keras.layers.recurrent import SimpleRNN
import numpy
from keras.utils import np_utils
n_hidden = 186
n_fac = 42
X = numpy.array(xData)  
Y = np_utils.to_categorical(yData) 
model=Sequential([
        Embedding(input_dim=X.shape[0], output_dim=n_fac, input_length=X.shape[1]),
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
        Dense(Y.shape[1], activation='softmax')
    ])


X2 = numpy.array(xData2)  
Y2 = np_utils.to_categorical(yData2) 
model2=Sequential([
        Embedding(input_dim=X2.shape[0], output_dim=n_fac, input_length=X2.shape[1]),
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
        Dense(Y2.shape[1], activation='softmax')
    ])


X3 = numpy.array(xData3) 
Y3 = np_utils.to_categorical(yData3) 
model3=Sequential([
        Embedding(input_dim=X3.shape[0], output_dim=n_fac, input_length=X3.shape[1]),
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
        Dense(Y3.shape[1], activation='softmax')
    ])



### Print the summary of the 3 models

In [10]:
model.summary()
model2.summary()
model3.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 5, 42)         108570      embedding_input_1[0][0]          
____________________________________________________________________________________________________
simplernn_1 (SimpleRNN)          (None, 186)           42594       embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 601)           112387      simplernn_1[0][0]                
Total params: 263,551
Trainable params: 263,551
Non-trainable params: 0
____________________________________________________________________________________________________
____________________________________________________________________________________________________
Layer (type)       

### Compile the models and fit them to the data for 100 epochs.

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model2.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model3.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [12]:
model.fit(X, y=Y, batch_size=200, nb_epoch=100, verbose=1)


Epoch 1/100
2585/2585 [==============================] - 0s - loss: 5.4738 - acc: 0.3110     
Epoch 2/100
2585/2585 [==============================] - 0s - loss: 4.8058 - acc: 0.3412     
Epoch 3/100
2585/2585 [==============================] - 0s - loss: 4.7751 - acc: 0.3412     
Epoch 4/100
2585/2585 [==============================] - 0s - loss: 4.7125 - acc: 0.3412     
Epoch 5/100
2585/2585 [==============================] - 0s - loss: 4.6778 - acc: 0.3412     
Epoch 6/100
2585/2585 [==============================] - 0s - loss: 4.7016 - acc: 0.3412     
Epoch 7/100
2585/2585 [==============================] - 0s - loss: 4.6526 - acc: 0.3412     
Epoch 8/100
2585/2585 [==============================] - 0s - loss: 4.6204 - acc: 0.3412     
Epoch 9/100
2585/2585 [==============================] - 0s - loss: 4.5910 - acc: 0.3412     
Epoch 10/100
2585/2585 [==============================] - 0s - loss: 4.5938 - acc: 0.3412     
Epoch 11/100
2585/2585 [==============================] - 0

In [13]:
model2.fit(X2, y=Y2, batch_size=200, nb_epoch=100, verbose=1)


Epoch 1/100
2823/2823 [==============================] - 0s - loss: 5.8394 - acc: 0.1923     
Epoch 2/100
2823/2823 [==============================] - 0s - loss: 5.3878 - acc: 0.2111     
Epoch 3/100
2823/2823 [==============================] - 0s - loss: 5.2779 - acc: 0.2111     
Epoch 4/100
2823/2823 [==============================] - 0s - loss: 5.2020 - acc: 0.2111     
Epoch 5/100
2823/2823 [==============================] - 0s - loss: 5.0985 - acc: 0.2129     
Epoch 6/100
2823/2823 [==============================] - 0s - loss: 4.9147 - acc: 0.2193     
Epoch 7/100
2823/2823 [==============================] - 0s - loss: 4.7981 - acc: 0.2196     
Epoch 8/100
2823/2823 [==============================] - 0s - loss: 4.6732 - acc: 0.2256     
Epoch 9/100
2823/2823 [==============================] - 0s - loss: 4.5498 - acc: 0.2299     
Epoch 10/100
2823/2823 [==============================] - 0s - loss: 4.4174 - acc: 0.2370     
Epoch 11/100
2823/2823 [==============================] - 0

In [14]:
model3.fit(X3, y=Y3, batch_size=200, nb_epoch=100, verbose=1)

Epoch 1/100
6234/6234 [==============================] - 0s - loss: 5.0379 - acc: 0.1833     
Epoch 2/100
6234/6234 [==============================] - 0s - loss: 4.7024 - acc: 0.1888     
Epoch 3/100
6234/6234 [==============================] - 0s - loss: 4.6433 - acc: 0.1888     
Epoch 4/100
6234/6234 [==============================] - 0s - loss: 4.5834 - acc: 0.1878     
Epoch 5/100
6234/6234 [==============================] - 0s - loss: 4.4776 - acc: 0.1893     
Epoch 6/100
6234/6234 [==============================] - 0s - loss: 4.3784 - acc: 0.1960     
Epoch 7/100
6234/6234 [==============================] - 0s - loss: 4.2683 - acc: 0.1970     
Epoch 8/100
6234/6234 [==============================] - 0s - loss: 4.1895 - acc: 0.1996     
Epoch 9/100
6234/6234 [==============================] - 0s - loss: 4.0878 - acc: 0.2056     
Epoch 10/100
6234/6234 [==============================] - 0s - loss: 3.9989 - acc: 0.2114     
Epoch 11/100
6234/6234 [==============================] - 0

## Genarate texts from each data using the models we created

In [15]:
answers = []
unk = convert_words_to_int["unknown"]
wikis = []
count_wiki = len(df[df.text == 'wiki'])
for j in range(0, int(0.3*count_wiki)):
    prev5 = np.array([convert_words_to_int[word] for word in vocab_words[j*10:j*10+5]])
    ans = ""
    for i in range(0, 100):
        bla = numpy.reshape(prev5, (1, len(prev5)))
        pred = model.predict(bla, verbose=0)
        word1 = pred.argsort()[0][-2:]
        # print (word1[0])
        if word1[0] == unk:
            word1 = word1[1]
        else:
            word1 = word1[0]
        # print(convert_int_to_words[word1])
        ans += convert_int_to_words[word1] + " "
        prev5 = prev5[1:5]
        prev5 = numpy.append(prev5, word1)
    wikis.append(ans)
    answers.append("wiki")

In [16]:
unk2 = convert_words_to_int2["unknown"]
cnns = []
count_cnn = len(df[df.text == 'cnn'])
for j in range(0, int(0.3*count_cnn)):
    prev6 = np.array([convert_words_to_int2[word] for word in vocab_words2[j*10:j*10+5]])
    ans2 = ""
    for i in range(0, 100):
        bla2 = numpy.reshape(prev6, (1, len(prev6)))
        pred2 = model2.predict(bla2, verbose=0)
        word2 = pred2.argsort()[0][-2:]
        if word2[0] == unk2:
            word2 = word2[1]
        else:
            word2 = word2[0]
        ans2 += convert_int_to_words2[word2] + " "
        prev6 = prev6[1:5]
        prev6 = numpy.append(prev6, word2)
    cnns.append(ans2)
    answers.append("cnn")

In [17]:
unk3 = convert_words_to_int3["unknown"]
recipes = []
count_recipe = len(df[df.text == 'recipe'])
for j in range(0, int(0.3*count_recipe)):
    prev7 = np.array([convert_words_to_int3[word] for word in vocab_words3[j*10:j*10+5]])
    ans3 = ""
    for i in range(0, 100):
        bla3 = numpy.reshape(prev7, (1, len(prev7)))
        pred3 = model3.predict(bla3, verbose=0)
        word3 = pred3.argsort()[0][-2:]
        if word3[0] == unk3:
            word3 = word3[1]
        else:
            word3 = word3[0]
        ans3 += convert_int_to_words3[word3] + " "
        prev7 = prev7[1:5]
        prev7 = numpy.append(prev7, word3)
    recipes.append(ans3)
    answers.append("recipe")

### now, we load our previous model from part b of the assignment and we will try to classify the data we created 

In [18]:
from sklearn.externals import joblib
import pickle
forest = joblib.load('forest.pkl')
vectorizer = None
with open('vectorizer.pkl', 'rb') as f:
    vectorizer_vocab = pickle.load(f)

In [19]:
import nltk
from nltk.corpus import stopwords # Import the stop word list
from sklearn.feature_extraction.text import CountVectorizer

def parse(text):
    words = nltk.word_tokenize(text) # Split into words
    words = [w for w in words if not w in stopwords.words("english")]
    porter = nltk.PorterStemmer()
    [porter.stem(w) for w in words]
    return " ".join(words)

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             vocabulary = vectorizer_vocab,
                             min_df = 10) 

cleaned_data = [parse(t) for t in wikis]
train_data_features = vectorizer.fit_transform(cleaned_data)
train_data_features = train_data_features.toarray()




In [20]:
cleaned_data2 = [parse(t) for t in cnns]
train_data_features2 = vectorizer.fit_transform(cleaned_data2)
train_data_features2 = train_data_features2.toarray()

In [21]:
cleaned_data3 = [parse(t) for t in recipes]
train_data_features3 = vectorizer.fit_transform(cleaned_data3)
train_data_features3 = train_data_features3.toarray()

## Merge data from cnn, wiki, recipes 

In [22]:
cleaned_data4 = cleaned_data + cleaned_data2 + cleaned_data3
train_data_features4 = vectorizer.fit_transform(cleaned_data4)
train_data_features4 = train_data_features4.toarray()

### check the score for each classification

In [23]:
forest.score(train_data_features, ['wiki']*len(cleaned_data))

0.80000000000000004

In [24]:
forest.score(train_data_features2, ['cnn']*len(cleaned_data2))

1.0

In [25]:
forest.score(train_data_features3, ['recipe']*len(cleaned_data3))

0.94117647058823528

In [26]:
final_ans = ["wiki"]*len(cleaned_data) + ["cnn"]*len(cleaned_data2) + ["recipe"]*len(cleaned_data3)

In [27]:
print("Accuracy: %0.2f" % forest.score(train_data_features4,final_ans))

Accuracy: 0.91


## wiki - we have some mistakes, maybe because the text is similar to some cnn articles because it has been choose randomaly
## cnn - all true
## recipe, small mistake

In [28]:
from sklearn.metrics import confusion_matrix
prediction = forest.predict(train_data_features4)
confusion_matrix(final_ans, prediction, labels=["wiki", "cnn","recipe"])

array([[12,  3,  0],
       [ 0, 15,  0],
       [ 0,  1, 16]])